In [ ]:
import os
import sys
sys.path.append('../')

from settings import config as cfg
from utils import io_utils as iou
from utils import waveform_plot_utils as wpu
from utils import fourier_math_utils as fmu

import pyaudio
import numpy as np
#import scipio
from scipy.io import wavfile
from scipy.io.wavfile import write
from io import BytesIO
from IPython.display import Audio, display, HTML
import matplotlib.pyplot as plt
from weasyprint import HTML
import sympy
import librosa
import pandas as pd
import re

from IPython.display import Audio
import ipywidgets as widgets

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from IPython.display import display, clear_output
import math
import plotly.io as pio

# Define functions

In [ ]:
# Utility function two display two audios side by side in the notebook
def audioSideBySide(name1, audio1, name2, audio2):
    text = '''
%s%s
%s%s
'''% (name1, name2, audio1._repr_html_(), audio2._repr_html_())
    display(HTML(text))

In [ ]:
def fourierSeries(period, N):
    """
    Calculate the Fourier series coefficients up to the Nth harmonic.

    This function computes the Fourier series coefficients for a given periodic signal
    represented as a discrete set of data points. It returns the coefficients (an, bn)
    for each harmonic up to the Nth, including the average term a0.

    The Fourier series of a periodic function f(t) with period T is given by:
    f(t) = a0 + ∑(an * cos(2 * pi * n * t / T) + bn * sin(2 * pi * n * t / T)),
    where:
    - a0 is the average value of the function over one period,
    - an and bn are the cosine and sine coefficients for the nth harmonic, respectively.

    The coefficients are calculated using the integrals:
    an = (2/T) * ∑(f(t) * cos(2 * pi * n * t / T))
    bn = (2/T) * ∑(f(t) * sin(2 * pi * n * t / T))

    Parameters:
        period (np.ndarray): An array of sampled values from one period of the periodic function.
        N (int): The number of harmonics to calculate, including the zeroth term.

    Returns:
        np.ndarray: An array of tuples, each containing the cosine and sine coefficients (an, bn)
                    for n ranging from 0 to N.

    Examples:
        >>> period = np.array([0, 1, 0, -1])  # Sampled values for one period of a square wave
        >>> fourierSeries(period, 3)
        array([[ 0. ,  0. ],
               [ 0. ,  1. ],
               [ 0. ,  0. ],
               [ 0. , -0.5]])
    """
    
    result = []
    
    T = len(period)
    t = np.arange(T)
    
    for n in range(N+1):
        
        an = 2/T*(period * np.cos(2*np.pi*n*t/T)).sum()
        bn = 2/T*(period * np.sin(2*np.pi*n*t/T)).sum()
        
        result.append((an, bn))
        
    return np.array(result)

Function that takes Fourier coefficients and reconstructs the original waveform

In [ ]:
def reconstruct(P, anbn):
    """Sum up sines and cosines according to the coefficients to 
    produce a reconstruction of the original waveform"""
    result = 0
    t = np.arange(P)
    for n, (a, b) in enumerate(anbn):
        if n == 0:
            a = a/2
        result = result + a*np.cos(2*np.pi*n*t/P) + b * np.sin(2*np.pi*n*t/P)
    return result

Function that takes just one period of the sound

In [ ]:
def extractPeriod(data, rate, t_start, t_end):
    t = np.arange(0,len(data))/rate
    plt.plot(t, data)

    duration = t_end - t_start
    plt.xlabel('$t$'); plt.ylabel('$x(t)$');


    sample_start = int(t_start * rate)
    sample_end = int(t_end*rate)

    period = data[sample_start:sample_end]
    #audioSideBySide("Original", display(Audio(data=data,rate=rate)), 
    #                "Extracted period", display(Audio(np.tile(period, int(1/duration)), rate=rate)))
    
    print("Original")
    display(Audio(data=data,rate=rate))
    print("Extracted period")
    display(Audio(np.tile(period, int(1/duration)), rate=rate))
    
    return period, rate

Function that tries to reconstruct the sound using N harmonics

In [ ]:
def approximateUpToNthHarmonic(period, rate, N, T, name):
    t = np.arange(len(period)) / rate
    duration = t.max()
    F = fourierSeries(period, N)
    #powers = np.sqrt(np.sum(F**2, axis=1))
    powers = np.sqrt(np.sum(F**2, axis=1))
    powers_sum = np.sum(powers)
    powers = powers / powers_sum
    np.set_printoptions(suppress=True) # suppress scientific notation
    powers = powers.round(4) # rounding to 4 decimal places   
    powers_print = powers*100
    powers_print = np.array(list(map(lambda x: "{:.4f}".format(x).replace('.',',').rstrip('0'), powers_print)))


    reconstructed = reconstruct(len(period), F)
    powers[0] = 0 # I don't want to see the zero-frequency cosine since it is just a constant signal
    
    functions = [''] * N
    #print("The function that generated the data in the 'reconstructed' array is:")
    #print("f(t) = ", end="")
    for n, (a, b) in enumerate(F):
        if n == 0:
            print(f"{a/2:.3f}", end="")
            function = f"{a/2:.3f}"
        else:
            print(f" + {a:.3f}*cos(2*pi*{n}*t/{T}) + {b:.3f}*sin(2*pi*{n}*t/{T})", end="")
            print()
            if functions[0] != "":
                function = f" + {a:.3f}*cos(2*pi*{n}*t/{T}) + {b:.3f}*sin(2*pi*{n}*t/{T})"
            else:
                function += f" + {a:.3f}*cos(2*pi*{n}*t/{T}) + {b:.3f}*sin(2*pi*{n}*t/{T})"
            functions[n-1] = function
        
    plt.subplots(figsize=(12,4)) #inače zakomentirano
    plt.subplot(121)             #inače zakomentirano
    #plt.xlabel('t [ms]', fontsize=16)
    #plt.ylabel('y(t)', fontsize=16)
    #plt.plot(t*1000, period, label='Original') 
    #plt.plot(t*1000, reconstructed, label='Reconstructed')
    plt.subplot(122)             #inače zakomentirano
    plt.bar(np.arange(len(powers)), powers)        #inače zakomentirano
    plt.xlabel('harmonic', fontsize=16)
    plt.ylabel('<P>', fontsize=16)                 #inače zakomentirano
    print(powers_print)
    #plt.savefig("./analysed/reco_n" + str(N) + "_" + name + ".pdf")
    
    print("Original")
    display(Audio(data=np.tile(period, int(0.7/duration)), rate=rate))
    print("Reconstructed up to %d harmonics" % N)
    display(Audio(data=np.tile(reconstructed, int(0.7/duration)), rate=rate))
    write("../results/analysed/reco_n" + str(N) + "_" + name + ".wav", rate, np.tile(reconstructed, int(0.7/duration)))
    
    return(F, functions, powers)

Function that draws each harmonic

In [ ]:
def draw_harmonics(functions, x_min, x_max, y_min, y_max, name, limit_N):
    
    for N, i in enumerate(functions):
    
        # parse the function string using sympy
        t = sympy.symbols('t')
        f = sympy.sympify(i)

        # create a list of t values within the specified range
        t_values = np.linspace(x_min, x_max, 1000)

        # evaluate the function for each value of t
        y_values = [f.evalf(subs={t: value}) for value in t_values]

        # plot the function using matplotlib
        plt.plot(t_values, y_values)

        # set the x and y axis limits
        plt.xlim(x_min, x_max)
        plt.ylim(y_min, y_max)

        # add labels and a title to the plot
        plt.xlabel('t [ms]', fontsize=16)
        plt.ylabel('y(t)', fontsize=16)
        plt.title('Plot of the function y(t)', fontsize=16)
        
        if N < limit_N:
            print("y(t) = " + str(f))
            print("y(t) = " + sympy.latex(f))
            plt.savefig("../results/analysed/f_harmonic_" + str(N+1) + "_" + name + ".pdf")
            plt.show()

Function that takes as a string mathematical function and extracts fourier coefficients and frequency

In [ ]:
def extract_a_b_f(function):
    a = b = frequency = 0
    try:
        # Extract the coefficient of the sine term
        match = re.search("(-?\d+\.\d+)[*]sin\((\d+\.\d+)[*]pi[*]t\)", function)
        if match:
            a = float(match.group(1))
            frequency = float(match.group(2))/2
        
        # Extract the coefficient of the cosine term
        match = re.search("(-?\d+\.\d+)[*]cos\((\d+\.\d+)[*]pi[*]t\)", function)
        if match:
            b = float(match.group(1))
            frequency = float(match.group(2))/2*1000
        
    except:
        pass
    return a, b, frequency

Function that produces the sound from the function in string format

In [ ]:
def produce_sound (functions, powers, name, limit_N):
    duration = 1.5
    total_power = np.sum(powers)
    samples_together = 0
    rate = 44100
    for i, func in enumerate(functions):
        t = sympy.symbols('t')
        function = sympy.sympify(func)
        a, b, f = extract_a_b_f(str(function))
        frequency = f
        amplitude = np.sqrt(a*a + b*b) * np.sqrt(powers[i+1] / total_power)
        t = np.linspace(0, duration, int(duration * rate))
        samples = amplitude * np.sin(2 * np.pi * frequency * t)
        if i >= 0:
            samples_together += samples
        if i < limit_N:
            audio = Audio(samples, rate=rate)
            display(audio)
            write(f"../results/analysed/f_harmonic_{i+1}_{name}.wav", rate, samples)

# Loading sound

In [ ]:
files = [os.path.join(cfg.PATH_INSTRUMENT_SAMPLES, name) for name in os.listdir(cfg.PATH_INSTRUMENT_SAMPLES)]
sounds = []

for file in files:
    path = os.path.join(cfg.PATH_INSTRUMENT_SAMPLES, file)
    sound, rate = iou.load_sound(path)
    sounds.append((sound, rate))

In [ ]:
iou.play_audio(files, n_columns=4)

In [ ]:
zoom_percentages = [0.008, 0.009, 0.01, 0.005, 0.005, 0.009, 0.003,
                    0.004, 0.015, 0.005, 0.015, 0.006, 0.009, 0.005, 
                    0.005, 0.005, 0.008]
wpu.plot_waveform(sounds, zoom_percentages, files)

# Extract one period

In [ ]:
periods = []
rates = []

name = "piano"
periods, data_rates = fmu.extract_periods_and_data_rates(sounds)




for n, (S, P) in enumerate(zip(sounds, periodBounds)): # (S, P) = data, rate, period_start, period_end
    plt.subplot(len(sounds), 1, n+1)
    period, rate = extractPeriod(S[0], S[1], P[0], P[1])
    periods.append(period); rates.append(rate)
    
    if name == 'cello':
        plt.axis([0.8260, 0.8380, -1, 1]) # cello
        plt.axvline(x=periodBounds[0][0], color = "red") #cello
        plt.axvline(x=periodBounds[0][1], color = "red") #cello
        
    if name == 'clarinet':
        plt.axis([2.088, 2.095, -1, 1])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'double_bass':
        plt.axis([0.6370,0.6480 , -1, 1])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'vocal':
        plt.axis([0.656, 0.666, -1, 1])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'flute':
        plt.axis([0.779, 0.7825, -1, 1])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'guitar_nylon':
        plt.axis([0.441, 0.447, -0.2, 0.2])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'oboe':
        plt.axis([0.546, 0.553, -1.2, 1.2])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'piano':
        plt.axis([0.757, 0.765, -0.3, 0.3])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'piccolo':
        plt.axis([0.6915, 0.6945, -1, 1])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'sax_alto':
        plt.axis([1.2625, 1.2665, -1, 1])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'sax_baritone':
        plt.axis([2.130, 2.160, -1, 1])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'sax_soprano':
        plt.axis([1.510, 1.520, -1, 1])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'sax_tenor':
        plt.axis([1.084, 1.093, -1, 1])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'guitar_metal':
        plt.axis([0.593, 0.601, -0.3, 0.3])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'trumpet':
        plt.axis([1.1275, 1.1315, -1, 1])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'violin':
        plt.axis([1.286, 1.291, -1, 1])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
        
    if name == 'trombone':
        plt.axis([0.547, 0.557, -1, 1])
        plt.axvline(x=periodBounds[0][0], color = "red")
        plt.axvline(x=periodBounds[0][1], color = "red")
    
    plt.xlabel('t [ms]', fontsize=16)
    plt.ylabel('y(t)', fontsize=16)
    
plt.savefig("../results/analysed/" + name + "_period.pdf")

note_period = (periodBounds[0][1] - periodBounds[0][0])*1000
print(note_period)
print("Frequency: " + str(1/note_period*1000))

# Reconstruct the sound up to N harmonics

In [ ]:
for P, R in zip(periods, rates):
    F, functions, powers = approximateUpToNthHarmonic(P, R, 15, note_period, "guitar_nylon") #returns coefficients of Fourier series and functions and makes plots


### Draw individual harmonics and write their function

In [ ]:
#draw_harmonics(functions, 0, 8, -1, 0.5, "cello", limit_N = 30)
#draw_harmonics(functions, 0, 10, -0.8, 0.8, "clarinet", limit_N = 10)
#draw_harmonics(functions, 0, 10, -1, 0.7, "double_bass", limit_N = 30)
#draw_harmonics(functions, 0, 20, -1, 1, "vocal", limit_N = 20)
#draw_harmonics(functions, 0, 20, -0.8, 0.8, "flute", limit_N = 10)
draw_harmonics(functions, 0, 4, -0.07, 0.07, "guitar_nylon", limit_N = 20)
#draw_harmonics(functions, 0, 20, -0.6, 0.6, "oboe", limit_N = 20)
#draw_harmonics(functions, 0, 20, -0.4, 0.4, "piano", limit_N = 20)
#draw_harmonics(functions, 0, 20, -0.8, 0.8, "piccolo", limit_N = 10)
#draw_harmonics(functions, 0, 20, -0.8, 0.8, "sax_alto", limit_N = 10)
#draw_harmonics(functions, 0, 20, -0.8, 0.8, "sax_baritone", limit_N = 35)
#draw_harmonics(functions, 0, 20, -0.8, 0.8, "sax_soprano", limit_N = 10)
#draw_harmonics(functions, 0, 20, -0.8, 0.8, "sax_tenor", limit_N = 18)
#draw_harmonics(functions, 0, 20, -0.1, 0.1, "guitar_metal", limit_N = 20)
#draw_harmonics(functions, 0, 20, -0.35, 0.35, "trumpet", limit_N = 15)
#draw_harmonics(functions, 0, 20, -0.9, 0.9, "violin", limit_N = 15)
#draw_harmonics(functions, 0, 20, -0.8, 0.8, "trombone", limit_N = 20)

### Find harmonics in sound and their relative power using librosa

In [ ]:
# Load the waveform from a wav file
#waveform, sample_rate = librosa.load('./instrument_samples/cello-c3_16bit.WAV')
waveform, sample_rate = librosa.load('../data/instrument_samples/clarinet-c5_16_bit.WAV')
waveform, sample_rate = librosa.load('../data/instrument_samples/double-bass-c3_16_bit.WAV')

# Compute the power spectrum of the waveform
power_spectrum = np.abs(np.fft.rfft(waveform))**2

# Determine the harmonic frequencies and their relative power
harmonic_frequencies = np.fft.rfftfreq(waveform.size, d=1/sample_rate)
harmonic_powers = power_spectrum / np.sum(power_spectrum)

# Select only frequencies with relative power > 0.001
#harmonics = harmonic_frequencies[harmonic_powers > 0.001]
#powers = harmonic_powers[harmonic_powers > 0.001]
harmonics = harmonic_frequencies[harmonic_frequencies < 1500]
powers_librosa = harmonic_powers[harmonic_frequencies < 1500]

# Plot the power spectrum
plt.plot(harmonics, powers_librosa)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Relative Power')
plt.show()

In [ ]:
df = pd.DataFrame({'frequency': harmonics, 'powers': powers_librosa})
df["harmonic"] = df['frequency']/(1/note_period*1000)
df['harmonic'] = df['harmonic'].astype(int)

In [ ]:
df = df.sort_values('powers', ascending=False)
df.head(20)

# Generate sound of each harmonic

In [ ]:
produce_sound(functions, powers, "sax_baritone", limit_N = 60)